<a href="https://colab.research.google.com/github/tanoManzo/mimic_attitude/blob/dev/zero_shot_mimic_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-shot models testing

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline # E2E automized 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import torch.nn.functional as F # toolbox: softmax, sigmoid, ReLu


In [ ]:
# model name from huggingface.co/model name_id:model_name
models_name = {
  'roberta':'cardiffnlp/twitter-roberta-base-sentiment-latest',
  'distilbert':'distilbert-base-uncased-finetuned-sst-2-english',
  'minilm':'microsoft/MiniLM-L12-H384-uncased',
  'bloom':'bigscience/bloom-560m'
}

# load tokenizer and model 
tokenizers = [AutoTokenizer.from_pretrained(model_name) for model_name in models_name.values()]
models= [AutoModelForSequenceClassification.from_pretrained(model_name) for model_name in models_name.values()]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df_notes = pd.read_json('/content/drive/MyDrive/NIH/Data/annotated_burnout/dina.jsonl', lines=True)
df_notes = df_notes.iloc[20:][['id','text','cats']].reset_index(drop=True)
df_notes

,id,text,cats
0,670,NPN addendum\n\n\n4. Completed 21day course of...,[Overall Positive Note]
1,671,Nursing NICU Note\n\n\nPt required frequent ba...,[Overall Negative Note]
2,672,Nursing NICU Note\n\n\n1. resp. O/Pt remains o...,[Overall Neutral Note]
3,673,Nursing progress note\n\n\n#1 O: [**Known last...,[Overall Positive Note]
4,674,NP NOTE\nProcedure Note: [** 1876**] removal\n...,[Overall Neutral Note]
...,...,...,...
218,888,NPN 1900-0700\n\n\n2. RESP: Infant remains ...,[Overall Neutral Note]
219,889,NICU NSG ADMIT NOTE\n\n28 [**1-2**] week femal...,[Overall Neutral Note]
220,890,Nursing NICU Note Cont.\n\n6 Alt. in CV status...,[Overall Neutral Note]
221,891,"NPN Days\n\n\n#1 ID: Temp stable, infant is ac...",[Overall Neutral Note]


In [ ]:
# select dataframe
df_to_infer = df_notes.copy()

# parameters to set 
total_notes = len(df_to_infer)
batch_size = 32
iterations = int(len(df_to_infer)/batch_size)

In [ ]:
l_scores = {k:[] for k in models_name.keys()}
l_labels = {k:[] for k in models_name.keys()}
for iter in range(iterations):
  

  idx_start = batch_size*iter     # first window sentence idx in the dataframe
  idx_end = batch_size*(iter+1)   # last window sentence idx in the dataframe
  
  # get row sentences for batch
  sentences = list(df_to_infer['text'].iloc[idx_start:idx_end].values)
 
  for (index, (key, value)) in enumerate(models_name.items()):
    
    # tokenized input for the model 
    tokenizer = tokenizers[index]
    batch = tokenizer(sentences, # list of sentence 
                    padding=True, # add if short 
                    truncation=True, # remove if long 
                    max_length=512, # sent length
                    return_tensors="pt" # to return pytorch tensor (NO for TF)
                    )
  
  
    
    model = models[index]
    with torch.no_grad():
      outputs = model(**batch)
      predictions = F.softmax(outputs.logits, dim=1)
      label_ids = torch.argmax(predictions, dim=1)

      labels = [model.config.id2label[label_id] for label_id in label_ids.tolist()]
      scores = [round(float(predictions[idx][label_id].item()),4) for idx, label_id in enumerate(label_ids.tolist())]
      l_labels[key].extend(labels)
      l_scores[key].extend(scores)

In [ ]:
l_labels, l_scores

({'roberta': ['neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'negative',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neut

In [ ]:
a = {'roberta': ['neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'negative',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'negative',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'positive',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral'],
  'distilbert': ['NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'POSITIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE',
   'NEGATIVE'],
  'minilm': ['LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0'],
  'bloom': ['LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_1',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0',
   'LABEL_0']}

In [ ]:
b = {'roberta': [0.8578,
   0.7101,
   0.8717,
   0.8311,
   0.7898,
   0.5407,
   0.8518,
   0.864,
   0.8696,
   0.856,
   0.9047,
   0.8461,
   0.8649,
   0.8571,
   0.8678,
   0.8971,
   0.8689,
   0.7279,
   0.8653,
   0.814,
   0.8701,
   0.8015,
   0.786,
   0.8081,
   0.8569,
   0.8609,
   0.6161,
   0.8257,
   0.7114,
   0.8502,
   0.8021,
   0.8961,
   0.8685,
   0.7957,
   0.8478,
   0.9025,
   0.8539,
   0.8723,
   0.689,
   0.9025,
   0.8451,
   0.8694,
   0.8376,
   0.856,
   0.9174,
   0.8398,
   0.7692,
   0.883,
   0.8499,
   0.8458,
   0.8337,
   0.8318,
   0.8,
   0.7039,
   0.8316,
   0.8707,
   0.8518,
   0.8668,
   0.7595,
   0.7961,
   0.8435,
   0.8403,
   0.8358,
   0.7953,
   0.771,
   0.9196,
   0.7924,
   0.8872,
   0.6723,
   0.7981,
   0.8567,
   0.883,
   0.6099,
   0.764,
   0.8815,
   0.8406,
   0.8922,
   0.8722,
   0.879,
   0.8428,
   0.9234,
   0.841,
   0.8901,
   0.838,
   0.8848,
   0.7714,
   0.8664,
   0.8915,
   0.8352,
   0.869,
   0.8163,
   0.8927,
   0.8362,
   0.7981,
   0.8884,
   0.896,
   0.8795,
   0.8886,
   0.887,
   0.8136,
   0.8037,
   0.8683,
   0.9074,
   0.8392,
   0.8473,
   0.8831,
   0.7392,
   0.8754,
   0.7897,
   0.7944,
   0.4953,
   0.7803,
   0.8678,
   0.8036,
   0.5954,
   0.8723,
   0.8642,
   0.8644,
   0.882,
   0.8246,
   0.8946,
   0.8178,
   0.8209,
   0.5495,
   0.8473,
   0.8769,
   0.7923,
   0.8616,
   0.8582,
   0.8779,
   0.7966,
   0.912,
   0.7393,
   0.8708,
   0.8431,
   0.8389,
   0.8626,
   0.592,
   0.8856,
   0.8857,
   0.8546,
   0.8539,
   0.8369,
   0.8345,
   0.8509,
   0.8822,
   0.8517,
   0.8312,
   0.5215,
   0.7367,
   0.8806,
   0.8937,
   0.8524,
   0.8882,
   0.8919,
   0.8871,
   0.9041,
   0.9098,
   0.9272,
   0.8153,
   0.8585,
   0.5029,
   0.8305,
   0.8516,
   0.8852,
   0.8399,
   0.8185,
   0.9381,
   0.8554,
   0.865,
   0.8438,
   0.7901,
   0.8828,
   0.8322,
   0.798,
   0.7811,
   0.6059,
   0.5696,
   0.8247,
   0.7575,
   0.807,
   0.8497,
   0.7242,
   0.8241,
   0.8062,
   0.8928,
   0.8527,
   0.8766,
   0.8735,
   0.8134,
   0.9006,
   0.8821],
  'distilbert': [0.9965,
   0.9971,
   0.9986,
   0.9791,
   0.9889,
   0.9047,
   0.9981,
   0.8415,
   0.9982,
   0.9911,
   0.9961,
   0.9973,
   0.998,
   0.9966,
   0.9985,
   0.9955,
   0.9975,
   0.9386,
   0.9956,
   0.9762,
   0.9897,
   0.9478,
   0.985,
   0.9935,
   0.9962,
   0.9928,
   0.9945,
   0.9934,
   0.9981,
   0.9942,
   0.9989,
   0.998,
   0.9969,
   0.9543,
   0.9971,
   0.9926,
   0.996,
   0.9954,
   0.9971,
   0.9821,
   0.9955,
   0.9601,
   0.994,
   0.9977,
   0.973,
   0.9918,
   0.9985,
   0.9987,
   0.7773,
   0.998,
   0.9947,
   0.9965,
   0.9972,
   0.9988,
   0.9842,
   0.9983,
   0.9989,
   0.9986,
   0.9894,
   0.9979,
   0.996,
   0.9846,
   0.9989,
   0.9945,
   0.9942,
   0.9464,
   0.9919,
   0.9954,
   0.9957,
   0.9958,
   0.9952,
   0.9948,
   0.9869,
   0.9976,
   0.9711,
   0.9977,
   0.9957,
   0.9944,
   0.9787,
   0.9942,
   0.9682,
   0.9973,
   0.9988,
   0.9886,
   0.9961,
   0.8154,
   0.9964,
   0.9977,
   0.6526,
   0.9979,
   0.916,
   0.9984,
   0.9982,
   0.9932,
   0.8138,
   0.9924,
   0.9975,
   0.9911,
   0.9987,
   0.9942,
   0.8683,
   0.9969,
   0.9982,
   0.9983,
   0.5632,
   0.9961,
   0.9985,
   0.9922,
   0.7729,
   0.9963,
   0.8416,
   0.9976,
   0.9884,
   0.9636,
   0.9847,
   0.9971,
   0.9943,
   0.6068,
   0.9988,
   0.9982,
   0.9969,
   0.9942,
   0.9978,
   0.637,
   0.9982,
   0.9982,
   0.9984,
   0.9948,
   0.9683,
   0.9993,
   0.9929,
   0.9875,
   0.9995,
   0.9121,
   0.9955,
   0.9984,
   0.9984,
   0.7269,
   0.999,
   0.9957,
   0.924,
   0.5617,
   0.9974,
   0.9948,
   0.9679,
   0.8074,
   0.9982,
   0.9988,
   0.9993,
   0.9974,
   0.9939,
   0.9985,
   0.998,
   0.9987,
   0.9386,
   0.9963,
   0.9983,
   0.5752,
   0.9937,
   0.9968,
   0.9974,
   0.9984,
   0.9979,
   0.9957,
   0.9949,
   0.9942,
   0.9984,
   0.9995,
   0.997,
   0.9961,
   0.9942,
   0.9992,
   0.9949,
   0.9962,
   0.9957,
   0.9972,
   0.9994,
   0.9937,
   0.9952,
   0.9969,
   0.9923,
   0.9977,
   0.9983,
   0.9967,
   0.992,
   0.9939,
   0.9874,
   0.9936,
   0.9987,
   0.9983,
   0.8649,
   0.7931],
  'minilm': [0.5013,
   0.5015,
   0.5015,
   0.5013,
   0.501,
   0.5015,
   0.5015,
   0.5011,
   0.5013,
   0.5013,
   0.5004,
   0.5016,
   0.5014,
   0.5019,
   0.5019,
   0.5018,
   0.5016,
   0.5009,
   0.5018,
   0.5013,
   0.5014,
   0.5011,
   0.5017,
   0.5016,
   0.5018,
   0.5016,
   0.5013,
   0.5005,
   0.5016,
   0.501,
   0.5017,
   0.5017,
   0.5016,
   0.5014,
   0.5017,
   0.5007,
   0.5017,
   0.5012,
   0.5008,
   0.5013,
   0.5017,
   0.501,
   0.5017,
   0.5015,
   0.5009,
   0.5015,
   0.5015,
   0.5016,
   0.501,
   0.5017,
   0.5016,
   0.5019,
   0.5016,
   0.5015,
   0.5017,
   0.5015,
   0.5019,
   0.5016,
   0.5012,
   0.5016,
   0.5015,
   0.5018,
   0.5019,
   0.5014,
   0.5013,
   0.5008,
   0.5015,
   0.5006,
   0.5017,
   0.5013,
   0.5015,
   0.5017,
   0.5011,
   0.5016,
   0.5015,
   0.5016,
   0.5014,
   0.5011,
   0.5009,
   0.5017,
   0.5007,
   0.5019,
   0.502,
   0.5017,
   0.501,
   0.5014,
   0.5012,
   0.5015,
   0.5011,
   0.5017,
   0.5011,
   0.5017,
   0.5021,
   0.5018,
   0.5007,
   0.5013,
   0.5017,
   0.5019,
   0.5016,
   0.5015,
   0.501,
   0.5015,
   0.501,
   0.5018,
   0.5006,
   0.5017,
   0.5018,
   0.5017,
   0.5013,
   0.5017,
   0.501,
   0.5015,
   0.5018,
   0.5009,
   0.5004,
   0.5014,
   0.5018,
   0.5012,
   0.5017,
   0.5019,
   0.5016,
   0.5015,
   0.5019,
   0.5011,
   0.5013,
   0.5018,
   0.5015,
   0.5013,
   0.5017,
   0.5014,
   0.5015,
   0.5012,
   0.5014,
   0.5013,
   0.5013,
   0.5016,
   0.5019,
   0.5013,
   0.5016,
   0.5011,
   0.5017,
   0.501,
   0.5016,
   0.5016,
   0.5013,
   0.5007,
   0.5017,
   0.5015,
   0.5009,
   0.5019,
   0.5015,
   0.5015,
   0.5014,
   0.5016,
   0.5013,
   0.5014,
   0.501,
   0.5004,
   0.5007,
   0.5014,
   0.5017,
   0.5005,
   0.5018,
   0.5018,
   0.5012,
   0.5016,
   0.5016,
   0.5003,
   0.5016,
   0.5017,
   0.5015,
   0.5015,
   0.5017,
   0.5016,
   0.5019,
   0.5014,
   0.5017,
   0.5016,
   0.5016,
   0.5017,
   0.5011,
   0.5018,
   0.5014,
   0.5014,
   0.5014,
   0.5016,
   0.501,
   0.5017,
   0.502,
   0.5013,
   0.5014,
   0.5008],
  'bloom': [1.0,
   1.0,
   1.0,
   1.0,
   0.9996,
   0.902,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9368,
   1.0,
   0.9979,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9918,
   1.0,
   0.9999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9986,
   1.0,
   0.9999,
   1.0,
   1.0,
   0.9997,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999,
   1.0,
   1.0,
   1.0,
   1.0,
   0.8078,
   1.0,
   1.0,
   1.0,
   0.9936,
   1.0,
   1.0,
   0.9726,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9945,
   1.0,
   0.9999,
   1.0,
   1.0,
   1.0,
   1.0,
   0.8655,
   1.0,
   1.0,
   0.9918,
   1.0,
   0.9994,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9507,
   1.0,
   0.9997,
   1.0,
   0.9995,
   1.0,
   1.0,
   1.0,
   0.9978,
   1.0,
   0.9251,
   1.0,
   1.0,
   0.9991,
   1.0,
   1.0,
   1.0,
   0.9991,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9997,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9939,
   1.0,
   1.0,
   1.0,
   0.9932,
   1.0,
   1.0,
   0.9999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9928,
   1.0,
   0.9999,
   1.0,
   1.0,
   0.9976,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.9999,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   0.6014,
   1.0,
   0.696,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0]}

In [ ]:
import json


json1 = json.dumps(a)
# open file for writing, "w" 
f = open("/content/drive/MyDrive/NIH/Data/results_burnout/label_zeroshot.json","w")
# write json object to file
f.write(json1)


json2 = json.dumps(b)
# open file for writing, "w" 
f = open("/content/drive/MyDrive/NIH/Data/results_burnout/score_zeroshot.json","w")
# write json object to file
f.write(json2)

5670